In [768]:
import torch as tor
import numpy as np
import pandas as pd

In [769]:
data = pd.read_csv('signal/data.csv')
inp = data.drop(columns=['subject ID', 'labels']).to_numpy().astype('float32')
tar = np.array([0.0 if val=='normal' else 500.0 for val in data.get('labels')]).astype('float32')
inp,tar

(array([[640., 636., 630., ...,  nan,  nan,  nan],
        [656., 640., 648., ...,  nan,  nan,  nan],
        [572., 564., 550., ...,  nan,  nan,  nan],
        ...,
        [660., 640., 662., ...,  nan,  nan,  nan],
        [748., 728., 744., ...,  nan,  nan,  nan],
        [850., 796., 816., ...,  nan,  nan,  nan]], dtype=float32),
 array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500.],
       dtype=float32))

In [770]:
nmean = np.nanmean(inp, axis=1)
idxs = np.where(np.isnan(inp))
inp[idxs] = np.take(nmean, idxs[0])

In [771]:
inputs = tor.from_numpy(inp)
targets = tor.from_numpy(tar)
print(inputs)
print(targets)

tensor([[640.0000, 636.0000, 630.0000,  ..., 672.6525, 672.6525, 672.6525],
        [656.0000, 640.0000, 648.0000,  ..., 673.5938, 673.5938, 673.5938],
        [572.0000, 564.0000, 550.0000,  ..., 572.6745, 572.6745, 572.6745],
        ...,
        [660.0000, 640.0000, 662.0000,  ..., 591.0847, 591.0847, 591.0847],
        [748.0000, 728.0000, 744.0000,  ..., 692.9908, 692.9908, 692.9908],
        [850.0000, 796.0000, 816.0000,  ..., 800.4389, 800.4389, 800.4389]])
tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500.])


In [772]:
w = tor.randn(1, inputs.shape[1], requires_grad=True)
b = tor.randn(1, requires_grad=True)
print(w.shape,b.shape)

torch.Size([1, 682]) torch.Size([1])


In [773]:
def model(x):
	return x @ w.t() + b
# print(x.shape, w.t().shape, b.shape)

In [774]:
preds = model(inputs)
preds

tensor([[ 2722.6187],
        [  402.6663],
        [ 2669.6733],
        [ 1116.3335],
        [ 1248.1371],
        [-1419.0552],
        [  468.1733],
        [ 1499.1056],
        [ 1054.4137],
        [ 3244.7092],
        [ 2080.7332],
        [ 3560.9919],
        [  784.8591],
        [ 4014.1396],
        [ 1555.5674],
        [ 1710.2533],
        [ 3388.4011],
        [ 3126.4163],
        [  796.0776],
        [ 2237.9006],
        [ 2438.1243],
        [ 2100.6233],
        [ 2437.4016],
        [ 2686.1111],
        [  490.7683],
        [ 4241.0298],
        [  820.5071],
        [ 1575.8617],
        [ 1303.8134],
        [ 2669.6733],
        [ -841.4063],
        [ -712.6915],
        [  886.2231],
        [ 1247.4097],
        [ 3640.2283],
        [ 1619.4357],
        [ 3177.8811],
        [ -252.5752],
        [ 1776.8501],
        [ 2804.3379],
        [ -823.9207],
        [ 1451.2012],
        [ 1017.7319],
        [ -697.8168],
        [ 3005.3115],
        [ 

In [775]:
print(targets)

tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500.])


In [776]:
def mse(t1, t2):
	diff = t1 - t2
	return tor.sum(diff ** 2) / diff.numel()

In [777]:
loss = mse(preds, targets)
loss

tensor(3586280.5000, grad_fn=<DivBackward0>)

In [778]:
for i in range(100):
	preds = model(inputs)
	loss = mse(preds, targets)
	loss.backward()
	with tor.no_grad():
		w-=w.grad * 1e-10
		b-=b.grad * 1e-10
		w.grad.zero_()
		b.grad.zero_()

In [779]:
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(1656270.8750, grad_fn=<DivBackward0>)

In [780]:
preds

tensor([[ 1.3775e+03],
        [-9.4324e+02],
        [ 1.5237e+03],
        [-4.4560e+02],
        [ 9.7102e+00],
        [-2.9152e+03],
        [-9.4935e+02],
        [-5.4649e+01],
        [-2.8419e+02],
        [ 1.8148e+03],
        [ 5.3967e+02],
        [ 1.9758e+03],
        [-6.6968e+02],
        [ 2.4739e+03],
        [ 2.4451e+01],
        [ 3.8382e+02],
        [ 2.0379e+03],
        [ 1.7129e+03],
        [-4.1996e+02],
        [ 4.7576e+02],
        [ 1.1431e+03],
        [ 7.8134e+02],
        [ 9.5483e+02],
        [ 1.1772e+03],
        [-7.7496e+02],
        [ 2.8173e+03],
        [-7.2367e+02],
        [ 2.4212e+02],
        [ 2.0297e+00],
        [ 1.5237e+03],
        [-2.1361e+03],
        [-1.9897e+03],
        [-4.1399e+02],
        [-1.3003e+01],
        [ 2.0650e+03],
        [ 3.2630e+02],
        [ 1.7856e+03],
        [-1.6674e+03],
        [ 2.5570e+02],
        [ 1.5261e+03],
        [-2.2353e+03],
        [ 9.1850e+01],
        [-9.8958e+01],
        [-1

In [781]:
targets

tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500.])